In [45]:
from langchain_community.llms import Ollama
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv
import os
load_dotenv()

True

In [15]:
model = Ollama(base_url="http://localhost:11434",model="llama3")

In [46]:
db_user=os.getenv("DB_USER")
db_password=os.getenv("DB_PASSWORD")
db_host=os.getenv("DB_HOST")
db_name=os.getenv("DB_NAME")
db=SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
print(db.dialect)

mysql


In [47]:
print(db.get_usable_table_names())

['signup', 'signuptwo']


In [48]:
db.run("SELECT * FROM bankmanagementsystem.signup")

"[('0', 'Aryan Raina', 'Kamal Raina', 'Feb 4, 2003', 'Male', 'aryanraina2021@gmail.com', 'Unmarried', 'Lane no.22 Block no.130 Flat no.18 Jagti', 'Jammu', '181221', 'Jammu and kashmir'), ('0', 'Rishika Raina', 'Kamal Raina', 'Jul 27, 2005', 'Female', 'rishikaraina2005@gmail.com', 'Unmarried', 'Lane no.22 block no.130 Flat no.18 Jagti', 'Jammu', '181221', 'J&K'), ('6571', 'Kamal Raina', 'Janki Nath Raina', 'Jul 9, 2024', 'Male', 'kamalraina1969@gmail.com', 'married', 'ad ', 'dddd', '181221', 'dinin'), ('4374', 'rajan', 'bhan', 'Jul 4, 2024', 'Male', 'rajan44@gmail.com', 'married', 'Deathvalley ', 'Secret', '9000000', 'Unknown'), ('5015', 'rajan bhan', 'rajat bhan', 'Jul 1, 2024', 'Male', 'rajanbhan420@gmail.com', 'married', 'deathvalley', 'unknown', '910200', 'unknown')]"

In [19]:
from langchain.chains import create_sql_query_chain

In [20]:
chain= create_sql_query_chain(model,db)

In [21]:
response=chain.invoke({"question":"How many males are there in bankmanagementsystem.signup table"})

In [39]:
print(response)

Here is the answer:

Question: How many males are there in bankmanagementsystem.signup table

SQLQuery:
```sql
SELECT COUNT(*) 
FROM signup 
WHERE gender = 'Male';
``


In [9]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query=QuerySQLDataBaseTool(db=db)
write_query= create_sql_query_chain(model,db)
chain= write_query | execute_query
chain.invoke({"question":"How many tables are there"})

'Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near \'Here is the answer:\\n\\nQuestion: How many tables are there\\nSQLQuery: SELECT COU...\' at line 1")\n[SQL: Here is the answer:\n\nQuestion: How many tables are there\nSQLQuery: SELECT COUNT(*) FROM information_schema.tables WHERE table_name IN (\'signup\', \'signuptwo]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

In [10]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [11]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query and only give SQL query without any other information so that SQL with take only query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

In [12]:
chain=(
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query")|execute_query
    )
    | answer_prompt
    | model
    |StrOutputParser()
)

In [33]:
chain.invoke({"question":"How many tables are there"})

"A SQL error!\n\nThe error message is indicating that there's a syntax issue in the SQL query. Specifically, it seems that the `SQLQuery` section of the question contains the actual SQL code, which is not enclosed in backticks (``) or quotes (`''`). This is causing MariaDB to throw an error.\n\nTo fix this, we can simply enclose the entire SQL query in a string, like so:\n```sql\nSELECT COUNT(DISTINCT TABLE_NAME) \nFROM INFORMATION_SCHEMA.TABLES \nWHERE TABLE_SCHEMA = 'default';\n```\nNow, let's run this corrected query and answer the user question!\n\nAccording to the SQL result (assuming it's correct), there are ... [drumroll] ... **[insert number here]** tables in total!"

In [24]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
toolkit=SQLDatabaseToolkit(db=db,llm=model)

In [25]:
tools=toolkit.get_tools()

In [26]:
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000296807A79E0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000296807A79E0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000296807A79E0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [40]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQL query to run without giving query into quotes and other tokens , then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [41]:
from langchain.agents import create_sql_agent

In [42]:
agent_executor= create_sql_agent(llm=model,toolkit=toolkit,system_message=system_message,verbose=True)

In [44]:
agent_executor.run("how many males are in signup table")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Let's get started.

Thought: Before answering the question, I need to know what tables exist in the database.
Action: sql_db_list_tables
Action Input: empty stringsignup, signuptwoThought: Now that I have the list of tables, I can query the schema of the relevant table.
Action: sql_db_schema
Action Input: signup, signuptwo
CREATE TABLE signup (
	formno VARCHAR(20), 
	name VARCHAR(20), 
	father_name VARCHAR(20), 
	dob VARCHAR(20), 
	gender VARCHAR(20), 
	email VARCHAR(40), 
	marital_status VARCHAR(20), 
	address VARCHAR(40), 
	city VARCHAR(20), 
	pincode VARCHAR(20), 
	state VARCHAR(20)
)COLLATE utf8mb4_general_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from signup table:
formno	name	father_name	dob	gender	email	marital_status	address	city	pincode	state
0	Aryan Raina	Kamal Raina	Feb 4, 2003	Male	aryanraina2021@gmail.com	Unmarried	Lane no.22 Block no.130 Flat no.18 Jagti	Jammu	181221	Jammu and kashmir
0	Rishika Raina	Kamal Raina	Jul 27, 2005	Female	rishikaraina2005@gmail.com	Unm

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `You want me to create a SQL query that counts the number of male signups. Here's the query:

```sql
SELECT COUNT(*) 
FROM signuptwo 
WHERE gender = 'Male';
```

This query will return the count of signups where the gender is "Male". Let me know if you have any other questions or need further assistance!`